<a href="https://colab.research.google.com/github/MohrezSheikh/A-Dynamic-Web-App-Using-Pre-trained-Transformer-Models-for-Sentiment-Analysis-and-Text-Summarization/blob/main/Sentiment_Analysis_and_Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hazm
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 59.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 25.2 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9297 sha256=e83577394c56f7dc001414ac07c83b30e7c259a06355dacdbc1c632205119e0e
  Stored in directory: /root/.cache/pip/wheels/bc/be/39/c37ad168eb2ff644c9685f52554440372129450f0b8ed203dd
Successfully built flashtext
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 55.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.0 MB/s eta 0:00:00
  Preparing metadata

In [2]:
import pandas as pd

df = pd.read_csv('data.csv')

df = df.drop(columns=['Score', 'Suggestion'])

df = df.applymap(str)

token_val = [str(i) for i in df['Text'].values]

In [4]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, raw_data):
        # Initialize the dataset with raw data
        self.data = raw_data

    def __len__(self):
        # Return the total number of samples in the dataset
        return len(self.data)

    def __getitem__(self, index):
        # Retrieve and return a specific sample and its associated label
        sample = self.data[index]
        # Perform any preprocessing or transformations on the sample if needed
        processed_sample = self.preprocess(sample)

        # Return the processed sample and its label
        return processed_sample

    def preprocess(self, sample):
        # Preprocess the sample (replace with your own preprocessing logic)
        return sample

In [5]:
raw_data = token_val  # Your raw data here
dataset = CustomDataset(raw_data)

In [6]:
dataset[1]

'خرید این محصول رو توصیه میکنم '

In [8]:
num_samples = len(dataset)
print("Total number of samples:", num_samples)

Total number of samples: 3261


In [11]:
from hazm import stopwords_list, word_tokenize

persian_stopwords = set(stopwords_list())

def remove_stopwords(text):
    tokens = word_tokenize(text)

    # Remove stop words
    tokens = [token for token in tokens if token not in persian_stopwords]

    # Rejoin tokens
    clean_text = ' '.join(tokens)
    return clean_text


# Original dataset
dataset = dataset

# Create a new list to store the modified dataset
modified_dataset = []

# Iterate over the dataset
for text in dataset:
    # Apply remove_stopwords function
    modified_text = remove_stopwords(text)

    # Append the modified text to the new list
    modified_dataset.append(modified_text)

# Assign the modified dataset to the original variable
dataset = modified_dataset

In [12]:
dataset[1]

'خرید محصول توصیه میکنم'

In [13]:
from transformers import pipeline, RobertaTokenizer

In [44]:
# tokenizer = RobertaTokenizer.from_pretrained('HooshvareLab/bert-fa-zwnj-base')
classifier = pipeline('sentiment-analysis', model='HooshvareLab/bert-fa-base-uncased-sentiment-digikala')

In [53]:
import pandas as pd

# Assuming you have an existing DataFrame called 'df'
df['sentiment_label'] = ""
df['sentiment_score'] = 0.0

for i, text in enumerate(dataset):
    modified_text = remove_stopwords(text)
    sentiment = classifier(modified_text, truncation=True, max_length=512)
    df.loc[i, 'sentiment_label'] = sentiment[0]['label']
    df.loc[i, 'sentiment_score'] = sentiment[0]['score']

In [55]:
df

,Text,sentiment_label,sentiment_score
0,این اولین تجربه من برای خرید ایفون هست امروز...,recommended,0.677589
1,خرید این محصول رو توصیه میکنم,recommended,0.978979
2,1 ساله این گوشی رو دارم هیچ نقطه ضعفی ازش ند...,recommended,0.923763
3,سلام خدمت دوستان این گوشی از همه نظر عالی کیف...,recommended,0.749044
4,سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...,recommended,0.959186
...,...,...,...
3256,کیفت ساختش خوبه تنوعش هم خوبه فقط مشکلش اینه...,recommended,0.819608
3257,خیلی خوب و به درد بخوره و اندازشم عالیه,recommended,0.914075
3258,عالیه خیلی دوسش دارم قیمتش یه کم زیاده ولی به ...,recommended,0.943462
3259,من این مینی فرز رو از دیحیکالا سفارش دادم و به...,recommended,0.983471


import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df, *['sentiment_score'], **{})
chart

import numpy as np
from google.colab import autoviz

def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  df.groupby(colname).size().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(df, *['sentiment_label'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df, *['sentiment_score'], **{})
chart

import numpy as np
from google.colab import autoviz

def violin_plot(df, value_colname, facet_colname, figscale=1, mpl_palette_name='Dark2', **kwargs):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (12 * figscale, 1.2 * figscale * len(df[facet_colname].unique()))
  plt.figure(figsize=figsize)
  sns.violinplot(df, x=value_colname, y=facet_colname, palette=mpl_palette_name, **kwargs)
  sns.despine(top=True, right=True, bottom=True, left=True)
  return autoviz.MplChart.from_current_mpl_state()

chart = violin_plot(df, *['sentiment_score', 'sentiment_label'], **{'inner': 'box'})
chart

In [56]:
from bertopic import BERTopic

topic_model = BERTopic(language='persian')

In [73]:
topics, probs = topic_model.fit_transform(dataset)

In [74]:
df2 = pd.DataFrame(topic_model.get_document_info(dataset))
df2.head()

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,تجربه خرید ایفون هست امروز ب دستم رسیده اسم طل...,-1,-1_خریدم_داره_دیجی_کیفیت,"[خریدم, داره, دیجی, کیفیت, کالا, دستگاه, هست, ...",[بتازگی خریدم چندباری ازش پخت پیتزا کباب تابه ...,خریدم - داره - دیجی - کیفیت - کالا - دستگاه - ...,0.000000,False
1,خرید محصول توصیه میکنم,11,11_میکنم_شک_خریدش_بخرید,"[میکنم, شک, خریدش, بخرید, پیشنهاد, حتما, نکنید...","[عالیه شک نکنید خریدش, پیشنهاد خریدم واقعا عال...",میکنم - شک - خریدش - بخرید - پیشنهاد - حتما - ...,0.971798,False
2,1 گوشی دارم نقطه ضعفی ازش ندیدم,15,15_اصلا_امروز_راضی_نکردم,"[اصلا, امروز, راضی, نکردم, کف, میکردم, ضعفی, خ...",[الان ده ها هندزفری خریدم خراب هندزفری سال خری...,اصلا - امروز - راضی - نکردم - کف - میکردم - ضع...,0.618991,False
3,سلام خدمت دوستان گوشی کیفیت ساخت سرعت کیفیت صد...,1,1_هدفون_صدا_صدای_گوش,"[هدفون, صدا, صدای, گوش, کیفیت, صداش, بیس, هست,...",[تکیه اعتبار برند سنهایزر اقدام خرید هدفون ارگ...,هدفون - صدا - صدای - گوش - کیفیت - صداش - بیس ...,1.000000,False
4,سلام دوستانی نگران شکستن صفحه نمایش بدونند صفح...,3,3_گوشی_تبلت_صفحه_کار,"[گوشی, تبلت, صفحه, کار, سامسونگ, نمایش, دوربین...",[محصول خرید کرده_بودم گوشی قبلیم نوت 5 انصافا ...,گوشی - تبلت - صفحه - کار - سامسونگ - نمایش - د...,0.860548,False


In [75]:
topic_model.visualize_barchart()

In [76]:
df.rename(columns = {'Text':'Document'}, inplace = True)

In [79]:
df['Document'] = df2['Document'].values

In [80]:
df_merge = pd.merge(df, df2, on="Document")
df_merge.head()

,Document,sentiment_label,sentiment_score,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,تجربه خرید ایفون هست امروز ب دستم رسیده اسم طل...,recommended,0.677589,-1,-1_خریدم_داره_دیجی_کیفیت,"[خریدم, داره, دیجی, کیفیت, کالا, دستگاه, هست, ...",[بتازگی خریدم چندباری ازش پخت پیتزا کباب تابه ...,خریدم - داره - دیجی - کیفیت - کالا - دستگاه - ...,0.000000,False
1,خرید محصول توصیه میکنم,recommended,0.978979,11,11_میکنم_شک_خریدش_بخرید,"[میکنم, شک, خریدش, بخرید, پیشنهاد, حتما, نکنید...","[عالیه شک نکنید خریدش, پیشنهاد خریدم واقعا عال...",میکنم - شک - خریدش - بخرید - پیشنهاد - حتما - ...,0.971798,False
2,1 گوشی دارم نقطه ضعفی ازش ندیدم,recommended,0.923763,15,15_اصلا_امروز_راضی_نکردم,"[اصلا, امروز, راضی, نکردم, کف, میکردم, ضعفی, خ...",[الان ده ها هندزفری خریدم خراب هندزفری سال خری...,اصلا - امروز - راضی - نکردم - کف - میکردم - ضع...,0.618991,False
3,سلام خدمت دوستان گوشی کیفیت ساخت سرعت کیفیت صد...,recommended,0.749044,1,1_هدفون_صدا_صدای_گوش,"[هدفون, صدا, صدای, گوش, کیفیت, صداش, بیس, هست,...",[تکیه اعتبار برند سنهایزر اقدام خرید هدفون ارگ...,هدفون - صدا - صدای - گوش - کیفیت - صداش - بیس ...,1.000000,False
4,سلام دوستانی نگران شکستن صفحه نمایش بدونند صفح...,recommended,0.959186,3,3_گوشی_تبلت_صفحه_کار,"[گوشی, تبلت, صفحه, کار, سامسونگ, نمایش, دوربین...",[محصول خرید کرده_بودم گوشی قبلیم نوت 5 انصافا ...,گوشی - تبلت - صفحه - کار - سامسونگ - نمایش - د...,0.860548,False


In [81]:
Fdf = df_merge.drop(columns=['Representative_document', 'Probability', 'Representative_Docs'])

In [83]:
Fdf

,Document,sentiment_label,sentiment_score,Topic,Name,Representation,Top_n_words
0,تجربه خرید ایفون هست امروز ب دستم رسیده اسم طل...,recommended,0.677589,-1,-1_خریدم_داره_دیجی_کیفیت,"[خریدم, داره, دیجی, کیفیت, کالا, دستگاه, هست, ...",خریدم - داره - دیجی - کیفیت - کالا - دستگاه - ...
1,خرید محصول توصیه میکنم,recommended,0.978979,11,11_میکنم_شک_خریدش_بخرید,"[میکنم, شک, خریدش, بخرید, پیشنهاد, حتما, نکنید...",میکنم - شک - خریدش - بخرید - پیشنهاد - حتما - ...
2,1 گوشی دارم نقطه ضعفی ازش ندیدم,recommended,0.923763,15,15_اصلا_امروز_راضی_نکردم,"[اصلا, امروز, راضی, نکردم, کف, میکردم, ضعفی, خ...",اصلا - امروز - راضی - نکردم - کف - میکردم - ضع...
3,سلام خدمت دوستان گوشی کیفیت ساخت سرعت کیفیت صد...,recommended,0.749044,1,1_هدفون_صدا_صدای_گوش,"[هدفون, صدا, صدای, گوش, کیفیت, صداش, بیس, هست,...",هدفون - صدا - صدای - گوش - کیفیت - صداش - بیس ...
4,سلام دوستانی نگران شکستن صفحه نمایش بدونند صفح...,recommended,0.959186,3,3_گوشی_تبلت_صفحه_کار,"[گوشی, تبلت, صفحه, کار, سامسونگ, نمایش, دوربین...",گوشی - تبلت - صفحه - کار - سامسونگ - نمایش - د...
...,...,...,...,...,...,...,...
3956,اینو پیشنهاد 205 تومن گرفتم عالیه پرقدرت ساخت ...,recommended,0.986435,-1,-1_خریدم_داره_دیجی_کیفیت,"[خریدم, داره, دیجی, کیفیت, کالا, دستگاه, هست, ...",خریدم - داره - دیجی - کیفیت - کالا - دستگاه - ...
3957,دریل واقعا قوی حرفه ای ساخت کشور روسیه,recommended,0.976953,21,21_ایرانی_قابلمه_کالای_خزر,"[ایرانی, قابلمه, کالای, خزر, تولیدات, پارس, بی...",ایرانی - قابلمه - کالای - خزر - تولیدات - پارس...
3958,کیفت ساختش خوبه تنوعش خوبه مشکلش اینه پیچ گوشت...,recommended,0.819608,22,22_کیف_کیسه_چمن_دریل,"[کیف, کیسه, چمن, دریل, اونی, میفته, بزرگتر, جا...",کیف - کیسه - چمن - دریل - اونی - میفته - بزرگت...
3959,مینی فرز دیحیکالا سفارش دادم دستم العاده قدرتم...,recommended,0.983471,-1,-1_خریدم_داره_دیجی_کیفیت,"[خریدم, داره, دیجی, کیفیت, کالا, دستگاه, هست, ...",خریدم - داره - دیجی - کیفیت - کالا - دستگاه - ...
